# Análisis de sentimientos

En el presente notebook, vamos a estudiar y experimentar diferentes técnicas para realizar análisis de sentimientos a diferentes textos.

Las técnicas que veremos serán las siguientes: Modelos basados en lexicones, VADER y aprendizaje supervisado.

**Conjunto de datos**

El conjunto de datos que utilizaremos para nuestro análisis estará compuesto por un subconjunto de 10,000 reseñas de películas de un dataset de 50,000 reseñas de películas de IMDB que están clasificadas como positivas o negativas.

Dataset: https://www.kaggle.com/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews

In [ ]:
# Importamos algunas librerías
import pandas as pd
import numpy as np

In [ ]:
# Cargamos nuestro conjunto de datos
df =  pd.read_csv("imdb_dataset.csv")
df.shape

In [ ]:
df.sample(5)

In [ ]:
# Número de reseñas positivas y negativas
df["sentiment"].value_counts()

## 1. Modelos no supervisados basados en lexicones

El léxico AFINN es uno de los léxicos más simples y populares que se pueden utilizar para el análisis de sentimientos. La versión actual es "AFINN-en-165" y contiene más de 3300 palabras con una puntuación de polaridad asociada a cada una de ellas.

In [ ]:
# Leemos el diccionario de lexicones en un dataframe
df_lexicons = pd.read_csv("https://raw.githubusercontent.com/fnielsen/afinn/master/afinn/data/AFINN-en-165.txt", sep='\t',
                          names=["word", "polarity"])
df_lexicons.sample(10)

Ahora vamos a ver la distribución de polaridades de los lexicones

In [ ]:
# Importamos seaborn para visualizar la distribución
import seaborn

seaborn.histplot(df_lexicons["polarity"])

Ahora vamos a realizar un preprocesamiento básico a nuestro conjunto de datos. Pasaremos las reseñas a minúscula. Luego, removeremos los tokens que son stopwords o contengan caracteres diferentes del alfabeto.

In [ ]:
# Importamos las librerías necesarias para el preprocesamiento
import nltk
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

# Descargamos los paquetes necesarios para el preprocesamiento
nltk.download("punkt")
nltk.download("stopwords")
nltk.download("wordnet")
nltk.download("omw-1.4")

In [ ]:
# Lista de stopwords
stopwords = stopwords.words("english")
# Añadimos el tag "br" a la lista de stopwords
stopwords.append("br")
# Instanciamos el lematizador
lemmatizer = WordNetLemmatizer()

def preprocess_text(text, flag_lemmatizer=False):
    tokens = []

    for w in word_tokenize(text):
        # Convertimos a minúsculas
        w = w.lower()

        # Solo se filtran los tokens que estén compuestos por caracteres del alfabeto y que no sean stopwords
        if ((w.isalpha()) and (w not in stopwords)):
            if flag_lemmatizer:
                tokens.append(lemmatizer.lemmatize(w))
            else:
                tokens.append(w)

    return ' '.join(tokens)

In [ ]:
df["review_preprocessed"] = df.review.apply(preprocess_text, flag_lemmatizer=False)
df.sample(5)

Ahora, por cada review preprocesado obtenedremos su polaridad, el cuál será la suma de las polaridades de las palabras.

In [ ]:
# Convertiremos nuestro dataframe de lexicones a un diccionario para que su acceso sea más sencillo
dic_lexicons = df_lexicons.set_index("word")["polarity"].to_dict()
list(dic_lexicons.items())[:10]

In [ ]:
%%time
review_polarities = []

# Para cada review
for review in df["review_preprocessed"]:
  polarity = 0
  # Para cada token
  for token in word_tokenize(review):
      # Obtenemos su polaridad, si no la encuentra en el diccionario de lexicones le asigna un valor 0.
      p = dic_lexicons.get(token, 0)
      # Acumula la polaridad
      polarity += p
  review_polarities.append(polarity)

df["lexicon_polarity"] = review_polarities

In [ ]:
df.sample(5)

In [ ]:
# Gráfico de la distribución de las polaridades
seaborn.displot(data=df["lexicon_polarity"], kde=True)

In [ ]:
# Ahora, definimos que todos los review con polaridad mayor a 4 serán positivos, caso contrario negativos
df["lexicon_sentiment"] = ["positive" if p > 4 else "negative" for p in df["lexicon_polarity"]]

In [ ]:
df.sample(5)

Y ahora obtendremos el "accuracy" entre los valores de sentimientos reales con los predichos a través del modelo basados en lexicones.

In [ ]:
# Importamos algunas librerías que nos ayudará al cálculo de algunas métricas
from sklearn.metrics import accuracy_score, classification_report

# Cálculo del "accuracy"
accuracy_score(df["sentiment"], df["lexicon_sentiment"])

In [ ]:
# Más métricas
print(classification_report(df["sentiment"], df["lexicon_sentiment"]))

AFINN tiene una librería desarrollada en Python con el que podemos obtener el score de polaridad directamente

In [ ]:
# Instalamos la librería
!pip install afinn

In [ ]:
# Importamos la librería
from afinn import Afinn

In [ ]:
# Instanciamos nuestro objeto que nos ayudará a calcular la polaridad
afn = Afinn()

In [ ]:
%%time
# Obtenemos la polaridad para cada review a través de Afinn
df["afinn_polarity"] = [afn.score(review) for review in df["review_preprocessed"]]

In [ ]:
df.sample(5)

In [ ]:
# Ahora, aplicamos el mismo concepto que que todos los review con polaridad mayor a 4 serán positivos, caso contrario negativos
df["afinn_sentiment"] = ["positive" if p > 4 else "negative" for p in df["afinn_polarity"]]

In [ ]:
# Cálculo del "accuracy" para Affin
accuracy_score(df["sentiment"], df["afinn_sentiment"])

Podemos ver que el accuracy obtenido a través de la biblioteca Affin es el mismo que el que obtuvimos cuando realizamos el cálculo manualmente a través del diccionario de polaridades de lexicones.

## 2. VADER

Valence Aware Dictionary for sEntiment Reasoning (VADER), es un algoritmo de procesamiento de lenguaje natural que combina un enfoque de léxico de sentimientos, así como reglas gramaticales y convenciones sintácticas para expresar la polaridad y la intensidad de los sentimientos.

A diferencia de AFINN, este diccionario léxico contiene frases como "bad ass" y "the bomb", asi también emoticones como ":-)" y acrónimos cargados de sentimiento como "ROFL" y "WTF".

El diccionario de léxico de Vader contiene alrededor de 7500 "tokens" de sentimiento en total y cada uno asociada a una número de polaridad.

Además de los léxicos de sentimientos, existen estructuras que son inherentemente neutrales pero que pueden cambiar la polaridad del sentimiento (como "no" y "pero") o modificar la intensidad de la oración completa como "muy" y "extremadamente". En VADER, los desarrolladores incorporaron varias reglas heurísticas que manejan los casos de puntuación, mayúsculas, adverbios, etc.

In [ ]:
# Instalamos la librería VADER
!pip install vaderSentiment

In [ ]:
# Importamos la librería
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

In [ ]:
# Instanciamos nuestro objeto analizador
analyzer = SentimentIntensityAnalyzer()

Veamos algunos ejemplos de calculo de polaridad:

In [ ]:
analyzer.polarity_scores("hello")
# compound va de -1 a +1 (muy negativo a muy positivo)

In [ ]:
analyzer.polarity_scores("hello :)")

In [ ]:
analyzer.polarity_scores("hello :/")

In [ ]:
analyzer.polarity_scores("hello :(")

In [ ]:
analyzer.polarity_scores("WTF you are doing")

In [ ]:
analyzer.polarity_scores("WTF you are doing :@")

Ahora, obtenedremos los valores de polaridad utilizando la librería VADER.

In [ ]:
%%time
# Obteniendo la polaridad
df["vader_polarity"] = [analyzer.polarity_scores(r)["compound"] for r in df["review"]]

In [ ]:
# Gráfico de la distribución de las polaridades
seaborn.displot(data=df["vader_polarity"], kde=True)

In [ ]:
df.sample(5)

In [ ]:
# Ahora, definimos que todos los review con polaridad mayor a 0.5 serán positivos, caso contrario negativos
df["vader_sentiment"] = ["positive" if p >= 0.5 else "negative" for p in df["vader_polarity"]]

In [ ]:
df.sample(5)

Y ahora obtendremos el "accuracy" entre los valores de sentimientos reales con los predichos a través del modelo VADER.

In [ ]:
# Cálculo del "accuracy"
accuracy_score(df["sentiment"], df["vader_sentiment"])

In [ ]:
# Más métricas
print(classification_report(df["sentiment"], df["vader_sentiment"]))

## 3. Aprendizaje Supervisado

Finalmente, utilizaremos un modelo de aprendizaje supervisado, en este caso un clasificador "Random Forest". En esta ocasión, preprocesaremos la información con el flag de lematización prendido, esto quiere decir que se lematizará las palabras del conjunto de datos antes de enviarselas al modelo.

In [ ]:
%%time
# Preprocesamos las reseñas pero en esta ocasión con el flag prendido para lemmatizar los tokens
df["review_preprocessed_lemmatize"] = np.vectorize(preprocess_text)(df["review"], flag_lemmatizer=True)

In [ ]:
df[["review", "review_preprocessed", "review_preprocessed_lemmatize"]]

Ahora dividiremos el dataset en conjuntos de datos de entrenamiento y pruebas.

In [ ]:
# Importamos la librería que nos ayudará a dividir los conjuntos de datos
from sklearn.model_selection import train_test_split

# Dividimos los conjuntos de datos
X_train, X_test, y_train, y_test = train_test_split(df["review_preprocessed_lemmatize"], df["sentiment"], test_size=0.05, random_state=2023)

Utilizaremos la técnica TF-IDF para la vectorización de los datos

In [ ]:
# Importamos TF-IDF
from sklearn.feature_extraction.text import TfidfVectorizer

# Vectorizamos
# min_df: ignora a los términos que aparecen en menos de un número de documentos
tfidf = TfidfVectorizer(tokenizer=word_tokenize, use_idf=True, min_df=3, max_features=5000).fit(X_train.values.tolist() + X_test.values.tolist())

# Obtenemos los vectores
X_train_tf = tfidf.transform(X_train.values).toarray()
X_test_tf = tfidf.transform(X_test.values).toarray()

# Dimensiones de las variables independientes
X_train_tf.shape, X_test_tf.shape

Ahora sí, entrenamos un modelo "Random Forest"

In [ ]:
# Importamos la librería "Random Forest"
from sklearn.ensemble import RandomForestClassifier

# Entrenamos
random_forest = RandomForestClassifier().fit(X_train_tf, y_train)

In [ ]:
# Obtenemos la predicciones de nuestro conjunto de pruebas
y_pred = random_forest.predict(X_test_tf)

In [ ]:
# Calculamos el accuracy entre los valores reales de la data de prueba con los valores predichos.
accuracy_score(y_test, y_pred)
# Nota: estos valores no son "100%" comparables con los 2 modelos que realizamos más arriba ya que, por un lado hemos lematizado los conjuntos
# de datos y otro que estamos calculando el score sobre un conjunto de datos de prueba (no fue visto en entrenamiento).

Probando el modelo con otras reseñas ...

In [ ]:
sentence = "The movie was great"
other_review = preprocess_text(sentence, flag_lemmatizer=True)
random_forest.predict(tfidf.transform([other_review]).toarray())

In [ ]:
sentence = "The movie was really bad"
other_review = preprocess_text(sentence, flag_lemmatizer=True)
random_forest.predict(tfidf.transform([other_review]).toarray())

In [ ]:
sentence = "Henry Cavill performance was really bad"
other_review = preprocess_text(sentence, flag_lemmatizer=True)
random_forest.predict(tfidf.transform([other_review]).toarray())

In [ ]:
sentence = "Cris Evans is a good actor"
other_review = preprocess_text(sentence, flag_lemmatizer=True)
random_forest.predict(tfidf.transform([other_review]).toarray())